In [1]:
import torch
from torch import nn as nn

import torch_xla
import torch_xla.core.xla_model as xm
from torch_xla import runtime as xr
from torch_xla._internal import tpu
from torch_xla.experimental.scan_layers import scan_layers
import torch_xla.distributed.spmd as xs
import numpy as np

import torch_xla.debug.metrics as met
from torch_xla.experimental.custom_kernel import flash_attention

xr.use_spmd()

n_devices = xr.global_runtime_device_count()
xs.set_global_mesh(xs.HybridMesh(
  ici_mesh_shape=(4, 1),
  dcn_mesh_shape=(1, 1),
  axis_names=("fsdp", "tensor"),
))


/workspaces/torch/pytorch/xla/torch_xla/__init__.py:251: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


In [2]:
class FakeAttention(torch.nn.Module):
  def __init__(self, num_head=4, hidden_dim=256):
    super(FakeAttention, self).__init__()
    self.num_head = num_head
    self.hidden_dim = hidden_dim
    # self.d_k = hidden_dim // num_head
    self.fc = nn.Linear(hidden_dim, hidden_dim)

  def forward(self, input):
    # query_states: [B, NUM_HEAD, SEQ_LEN, d_k]
    # attn_output: [B, SEQ_LEN, d_m], dm = dk * NUM_HEAD
    query_states = input.clone()
    key_states = input.clone()
    value_states = input.clone()
    attn_output = flash_attention(
        query_states,
        key_states,
        value_states,
        causal=True,
        partition_spec=("fsdp", "tensor", None, None),
      )
    # print(attn_output.shape)
    # B, SEQ_LEN, d_m = attn_output.shape
    # output = attn_output.reshape(B, SEQ_LEN, NUM_HEAD, self.d_k).permute(0, 2, 1, 3)
    # attn_output = self.fc(attn_output)
    return attn_output


class DummyModule(torch.nn.Module):
  def __init__(self, num_layer=3, scan: bool = False):
    super(DummyModule, self).__init__()
    self.num_layer = num_layer
    self.layers = nn.ModuleList([FakeAttention() for i in range(self.num_layer)])
    self.scan = scan
  def forward(self, input):
    hidden_states = input
    xs.mark_sharding(hidden_states, xs.get_global_mesh(), ("fsdp", "tensor", None, None))
    if self.scan:
      hidden_states = scan_layers(self.layers, input_data = hidden_states)
    else:
      for layer in self.layers:
        hidden_states = layer(hidden_states)
    return hidden_states


In [ ]:
@torch_xla.compile(full_graph=False)
def test_fa_layers(scan: bool):
  with xm.xla_device():
    dm = DummyModule(3, scan=scan)
    hidden_states= torch.randn((2, 4, 256, 256)).requires_grad_()
  output = dm(hidden_states)
  loss = output.sum()
  loss.backward()
  print(hidden_states.grad)
  # print(output)

: 

In [ ]:
test_fa_layers(scan=False)